In [34]:
import win32api, win32con, win32gui
import ctypes
import pygetwindow as gw
from PIL import Image, ImageGrab
import cv2
import numpy as np
import pandas as pd
import pyautogui
from functions import *
import time
from random import randrange
import itertools
import math

In [23]:
window_size, window_top_left = obtain_window_info()
im = take_snippet(window_size,  window_top_left)
tile_number = convert_to_numbers(im)

In [24]:
unrevealed_tile = np.where(tile_number == 9)

In [25]:
linked_groups = [] # list of lists, with each inner list representing the index of the relevant numbered tile
# tile index starts with 0

# search through unrevealed_tiles to find the linked groups
for i in range(len(unrevealed_tile[0])):
    square = tile_number[unrevealed_tile[0][i]-1:unrevealed_tile[0][i]+2,
                         unrevealed_tile[1][i]-1:unrevealed_tile[1][i]+2
                        ]
    neighbor_nbr_tile = np.where((square >= 1) & (square <= 8))

    if len(neighbor_nbr_tile[0]) > 1:
        linked_group = []
        index_i = unrevealed_tile[0][i] * tile_number.shape[1] + unrevealed_tile[1][i]
        for j in range(len(neighbor_nbr_tile[0])):
            index_j = ((neighbor_nbr_tile[0][j] - 1) * tile_number.shape[1]) + \
                      (neighbor_nbr_tile[1][j] - 1) + index_i
            linked_group.append(index_j)
        linked_groups.append(linked_group)

In [26]:
# process the linked groups to get the exhausted linked pairs
linked_pairs = []
for i in linked_groups:
    if len(i) == 2:
        if i not in linked_pairs:
            linked_pairs.append(i)
    if len(i) > 2:
        for j in itertools.combinations(i, 2):
            k = [j[0], j[1]]
            if k not in linked_pairs:
                linked_pairs.append(k)

In [139]:
i = 0
target_group = linked_pairs[0]
element_count = len(target_group)
cashe_element_unrevealed_tiles = []
target_explosive_tiles_count = []
for j in target_group:
    ### for each element within a group, find all its nbr unrevealed tiles
    # find the height and width of the element
    height_j = math.floor(j/tile_number.shape[1])
    width_j = j % tile_number.shape[1]
    # extract the nbr 8 tiles
    square = tile_number[height_j-1:height_j+2, width_j-1:width_j+2]
    # find the nbr unrevealed tiles and extract their indices
    nbr_unrevealed_tiles = np.where(square == 9)
    nbr_unrevealed_indices = ((nbr_unrevealed_tiles[0] - 1) * tile_number.shape[1]) + \
                      (nbr_unrevealed_tiles[1] - 1) + j
    # cashe the nbr unrevealed tile indices
    cashe_element_unrevealed_tiles.append(nbr_unrevealed_indices)
    
    ### fine the target number of explosives for each element
    target_number = tile_number[height_j, width_j] - len(np.where(square == 11)[0])
    target_explosive_tiles_count.append(target_number)

### get the unique nbr unrevealed tile indices
unique_nbr_unrevealed_indices = [index for element in cashe_element_unrevealed_tiles \
                                 for index in element]
unique_nbr_unrevealed_indices = list(set(unique_nbr_unrevealed_indices))
unique_nbr_unrevealed_indices.sort()

### convert the indices in cashe_element_unrevealed_tiles from the indices of the map 
#   to indices (or positions to make distinguishing easier) 
#   of the unique_nbr_unrevealed_indices
cashe_element_unrevealed_pos = []
for j in cashe_element_unrevealed_tiles:
    temp = []
    for k in j:
        temp.append(unique_nbr_unrevealed_indices.index(k))    
    cashe_element_unrevealed_pos.append(temp)

### get all combination of 1's and 0's to represent whether or not each unreavealed tile is explosive 
explosive_combinations = list(itertools.product([0, 1], repeat=len(unique_nbr_unrevealed_indices)))

scenorios = []
### generate the number of neighboring bomb for each element for each scenario
# for idx,val in enumerate(explosive_combinations):
for scenario in explosive_combinations:
    temp = []
    for j in cashe_element_unrevealed_pos:
        explosive_count = sum([scenario[index] for index in j])
        temp.append(explosive_count)
    scenorios.append(temp)

In [128]:
jj = explosive_combinations[50]

In [131]:
selected_elements = [jj[index] for index in [0, 1, 2, 4, 5]]
# jj[[0, 1, 2, 4, 5]]

In [140]:
target_explosive_tiles_count

[2.0, 1.0]

In [135]:
scenorios

[[0, 0],
 [1, 0],
 [1, 0],
 [2, 0],
 [0, 1],
 [1, 1],
 [1, 1],
 [2, 1],
 [1, 1],
 [2, 1],
 [2, 1],
 [3, 1],
 [1, 2],
 [2, 2],
 [2, 2],
 [3, 2],
 [1, 1],
 [2, 1],
 [2, 1],
 [3, 1],
 [1, 2],
 [2, 2],
 [2, 2],
 [3, 2],
 [2, 2],
 [3, 2],
 [3, 2],
 [4, 2],
 [2, 3],
 [3, 3],
 [3, 3],
 [4, 3],
 [1, 0],
 [2, 0],
 [2, 0],
 [3, 0],
 [1, 1],
 [2, 1],
 [2, 1],
 [3, 1],
 [2, 1],
 [3, 1],
 [3, 1],
 [4, 1],
 [2, 2],
 [3, 2],
 [3, 2],
 [4, 2],
 [2, 1],
 [3, 1],
 [3, 1],
 [4, 1],
 [2, 2],
 [3, 2],
 [3, 2],
 [4, 2],
 [3, 2],
 [4, 2],
 [4, 2],
 [5, 2],
 [3, 3],
 [4, 3],
 [4, 3],
 [5, 3]]

In [132]:
sum(selected_elements)

3

In [111]:
cashe_element_unrevealed_pos

[[0, 1, 2, 4, 5], [1, 2, 3]]

In [93]:
unique_nbr_unrevealed_indices

[371, 372, 373, 374, 403, 435]

In [96]:
unique_nbr_unrevealed_indices.index(372)

1

In [84]:
explosive_tiles_count

[2.0, 1.0]

In [76]:
unique_nbr_unrevealed_indices

[371, 372, 373, 374, 403, 435]

In [55]:
cashe_element_unrevealed_tiles

[array([371, 372, 373, 403, 435], dtype=int64),
 array([372, 373, 374], dtype=int64)]

In [91]:
explosive_combinations

[(0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 1),
 (0, 0, 0, 0, 1, 0),
 (0, 0, 0, 0, 1, 1),
 (0, 0, 0, 1, 0, 0),
 (0, 0, 0, 1, 0, 1),
 (0, 0, 0, 1, 1, 0),
 (0, 0, 0, 1, 1, 1),
 (0, 0, 1, 0, 0, 0),
 (0, 0, 1, 0, 0, 1),
 (0, 0, 1, 0, 1, 0),
 (0, 0, 1, 0, 1, 1),
 (0, 0, 1, 1, 0, 0),
 (0, 0, 1, 1, 0, 1),
 (0, 0, 1, 1, 1, 0),
 (0, 0, 1, 1, 1, 1),
 (0, 1, 0, 0, 0, 0),
 (0, 1, 0, 0, 0, 1),
 (0, 1, 0, 0, 1, 0),
 (0, 1, 0, 0, 1, 1),
 (0, 1, 0, 1, 0, 0),
 (0, 1, 0, 1, 0, 1),
 (0, 1, 0, 1, 1, 0),
 (0, 1, 0, 1, 1, 1),
 (0, 1, 1, 0, 0, 0),
 (0, 1, 1, 0, 0, 1),
 (0, 1, 1, 0, 1, 0),
 (0, 1, 1, 0, 1, 1),
 (0, 1, 1, 1, 0, 0),
 (0, 1, 1, 1, 0, 1),
 (0, 1, 1, 1, 1, 0),
 (0, 1, 1, 1, 1, 1),
 (1, 0, 0, 0, 0, 0),
 (1, 0, 0, 0, 0, 1),
 (1, 0, 0, 0, 1, 0),
 (1, 0, 0, 0, 1, 1),
 (1, 0, 0, 1, 0, 0),
 (1, 0, 0, 1, 0, 1),
 (1, 0, 0, 1, 1, 0),
 (1, 0, 0, 1, 1, 1),
 (1, 0, 1, 0, 0, 0),
 (1, 0, 1, 0, 0, 1),
 (1, 0, 1, 0, 1, 0),
 (1, 0, 1, 0, 1, 1),
 (1, 0, 1, 1, 0, 0),
 (1, 0, 1, 1, 0, 1),
 (1, 0, 1, 1, 1, 0),
 (1, 0, 1, 1,

In [43]:
square = tile_number[height_j-1:height_j+2, width_j-1:width_j+2]

In [44]:
square

array([[9., 9., 9.],
       [9., 2., 1.],
       [9., 1., 0.]])

In [50]:
neighbor_unrevealed_tiles = np.where(square == 9)

In [48]:
neighbor_unrevealed_tiles

(array([0, 0, 0, 1, 2], dtype=int64), array([0, 1, 2, 0, 0], dtype=int64))

In [51]:
neighbor_unrevealed_indices = ((neighbor_unrevealed_tiles[0] - 1) * tile_number.shape[1]) + \
                      (neighbor_unrevealed_tiles[1] - 1) + j

In [52]:
neighbor_unrevealed_indices

array([371, 372, 373, 403, 435], dtype=int64)

In [29]:
target_group

[404, 405]

In [53]:
tile_number.shape

(18, 32)

In [100]:
itertools.combinations(stuff, L)

In [90]:
linked_groups

[[265, 266],
 [266, 267, 268],
 [267, 268, 269],
 [269, 270, 271],
 [261, 293],
 [272, 304],
 [293, 325, 357],
 [304, 336],
 [336, 367, 399],
 [389, 421],
 [399, 431],
 [421, 422, 487],
 [487, 519],
 [494, 526]]

In [59]:
square[0][j]

9.0

In [63]:
print(neighbor_nbr_tile[0][j])
print(neighbor_nbr_tile[1][j])

2
2


In [73]:
print(unrevealed_tile[0][153])
print(unrevealed_tile[1][153])

7
11


In [55]:
tile_number[unrevealed_tile[0][i], unrevealed_tile[1][i]]

9.0

In [69]:
i = 233
print(unrevealed_tile[0][i])
print(unrevealed_tile[1][i])

12
28


In [71]:
square

array([[12.,  9., 12.],
       [ 9.,  9., 12.],
       [10., 10., 10.]])

In [70]:
neighbor_nbr_tile

(array([], dtype=int64), array([], dtype=int64))

In [45]:
cashed_list

[[152, 0],
 [152, 1],
 [153, 0],
 [153, 1],
 [153, 2],
 [154, 0],
 [154, 1],
 [154, 2],
 [155, 0],
 [155, 1],
 [155, 2],
 [165, 0],
 [165, 1],
 [166, 0],
 [166, 1],
 [194, 0],
 [194, 1],
 [194, 2],
 [195, 0],
 [195, 1],
 [211, 0],
 [211, 1],
 [211, 2],
 [239, 0],
 [239, 1],
 [240, 0],
 [240, 1],
 [256, 0],
 [256, 1],
 [256, 2],
 [273, 0],
 [273, 1],
 [274, 0],
 [274, 1]]

In [53]:
tile_number.shape[1]

32

In [29]:
i = 152
square = tile_number[unrevealed_tile[0][i]-1:unrevealed_tile[0][i]+2,
                         unrevealed_tile[1][i]-1:unrevealed_tile[1][i]+2
                        ]


In [30]:
square

array([[ 9.,  9.,  9.],
       [12.,  9., 12.],
       [11.,  3.,  1.]])

In [14]:
len(np.where((square >= 1) & (square <= 8))[0])

9

In [22]:
tile_number

array([[10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10.],
       [10.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,
         9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,
         9.,  9.,  9.,  9.,  9., 10.],
       [10.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,
         9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,
         9.,  9.,  9.,  9.,  9., 10.],
       [10.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,
         9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,
         9.,  9.,  9.,  9.,  9., 10.],
       [10.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,
         9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,
         9.,  9.,  9.,  9.,  9., 10.],
       [10.,  9.,  9.,  2.,  2.,  3.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,
  

In [21]:
tile_number[unrevealed_tile[0][92], unrevealed_tile[1][92]]

9.0

In [18]:
unrevealed_tile[1][92]

3

In [19]:
unrevealed_tile

(array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,
         3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
         3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
         5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,
         6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
         6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
         7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,
         9, 